In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [28]:
%%capture --no-stderr
!pip install python-dotenv langchain-community langchain-core langchain langchain-openai langchain-chroma

In [29]:
# 라이브러리 불러오기
from dotenv import load_dotenv
import os
from langchain_openai import OpenAI

In [ ]:
# .env 파일에서 환경 변수 로드 (.env 파일에는 OPENAI API 키값을 적으면 됩니다. -> OPENAI_API_KEY=...)
load_dotenv("/content/.env")
# 환경 변수에서 API 키 가져오기
api_key = os.getenv("OPENAI_API_KEY")

In [30]:
llm = OpenAI(api_key=api_key)

In [45]:
# <`get_format_instructions` 메서드>

# 라이브러리 불러오기
from langchain_core.output_parsers import JsonOutputParser
# Json 출력 파서 불러오기
parser = JsonOutputParser()
instructions = parser.get_format_instructions()
print(instructions)  # JSON 형식의 지침을 출력

Return a JSON object.


In [46]:
# <`parse` 메서드>

ai_response = '{"이름": "김철수", "나이": 30}'
parsed_response = parser.parse(ai_response)
print(parsed_response)

{'이름': '김철수', '나이': 30}


랭체인 1.0에서는 `from langchain.output_parsers import RetryWithErrorOutputParser`가 제거되었습니다. 따라서 자체 구현합니다.

In [56]:
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.exceptions import OutputParserException
from langchain_openai import ChatOpenAI

class RetryWithErrorOutputParser:
    def __init__(self, parser, llm, max_retries=1):
        self.parser = parser
        self.llm = llm
        self.max_retries = max_retries
        self.retry_prompt = PromptTemplate.from_template(
            "Prompt:\n{prompt}\nCompletion:\n{completion}\n\n"
            "Above, the Completion did not satisfy the constraints given in the Prompt.\n"
            "Details: {error}\nPlease try again:"
        )
        self.retry_chain = self.retry_prompt | self.llm | StrOutputParser()

    @classmethod
    def from_llm(cls, parser, llm, max_retries=1):
        return cls(parser=parser, llm=llm, max_retries=max_retries)

    def parse_with_prompt(self, completion, prompt_value):
        # 원본처럼 to_string() 호출 (문자열이면 에러 발생)
        prompt_str = prompt_value.to_string()

        retries = 0
        while retries <= self.max_retries:
            try:
                return self.parser.parse(completion)
            except Exception as e:
                if retries == self.max_retries:
                    raise e
                retries += 1
                completion = self.retry_chain.invoke({
                    "prompt": prompt_str,
                    "completion": completion,
                    "error": repr(e)
                })
        raise OutputParserException("Failed to parse")


# 사용 예시
parser = RetryWithErrorOutputParser.from_llm(parser=JsonOutputParser(), llm=ChatOpenAI())

question = "가장 큰 대륙은?"
ai_response = "아시아입니다."

try:
    result = parser.parse_with_prompt(ai_response, question)
    print(result)
except Exception as e:
    print(f"오류 발생: {e}")

오류 발생: 'str' object has no attribute 'to_string'


In [57]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field, model_validator

# OpenAI 모델 설정
model = ChatOpenAI(model_name="gpt-4o", temperature=0.0)

# 원하는 데이터 구조 정의
class FinancialAdvice(BaseModel):
    setup: str = Field(description="금융 조언 상황을 설정하기 위한 질문")
    advice: str = Field(description="질문을 해결하기 위한 금융 답변")
    # Pydantic을 사용한 사용자 정의 검증 로직
    @model_validator(mode="before")
    @classmethod
    def question_ends_with_question_mark(cls, values: dict) -> dict:
        setup = values.get("setup", "")
        if not setup.endswith("?"):
            raise ValueError("잘못된 질문 형식입니다! 질문은 '?'로 끝나야 합니다.")
        return values

# 파서 설정 및 프롬프트 템플릿에 지침 삽입
parser = PydanticOutputParser(pydantic_object=FinancialAdvice)
prompt = PromptTemplate(
    template="다음 금융 관련 질문에 답변해 주세요.\n{format_instructions}\n질문: {query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
# 언어 모델을 사용해 데이터 구조를 채우도록 프롬프트와 모델 설정
chain = prompt | model | parser


# 체인 실행 및 결과 출력
try:
    result = chain.invoke({"query": "부동산에 관련하여 금융 조언을 받을 수 있는 질문하여라."})
    print(result)
except Exception as e:
    print(f"오류 발생: {e}")

setup='부동산 투자를 고려하고 있는데, 현재 시장 상황에서 어떤 전략이 가장 효과적일까요?' advice='현재 부동산 시장은 지역에 따라 다르게 움직일 수 있으므로, 먼저 관심 있는 지역의 시장 동향을 조사하는 것이 중요합니다. 또한, 장기적인 투자 목표를 설정하고, 이를 기반으로 리스크를 분산시키는 포트폴리오를 구성하는 것이 좋습니다. 전문가와 상담하여 세금 혜택이나 금융 상품을 활용하는 방법도 고려해 보세요.'


랭체인 1.0에서는 `from langchain.output_parsers.json import SimpleJsonOutputParser` 대신 `from langchain_core.output_parsers import JsonOutputParser as SimpleJsonOutputParser`를 사용합니다.

In [59]:
from langchain_core.output_parsers import JsonOutputParser as SimpleJsonOutputParser
# JSON 포맷의 응답을 생성하는 프롬프트 템플릿 설정
json_prompt = PromptTemplate.from_template(
    "다음 질문에 대한 답변이 포함된 JSON 객체를 반환하십시오: {question}"
)
json_parser = SimpleJsonOutputParser()
json_chain = json_prompt | model | json_parser

# 스트리밍 예시: 질문에 대한 답변이 부분적으로 구문 분석됨
list(json_chain.stream({"question": "비트코인에 대한 짧은 한문장 설명."}))

[{},
 {'description': ''},
 {'description': '비'},
 {'description': '비트'},
 {'description': '비트코'},
 {'description': '비트코인은'},
 {'description': '비트코인은 분'},
 {'description': '비트코인은 분산'},
 {'description': '비트코인은 분산된'},
 {'description': '비트코인은 분산된 디'},
 {'description': '비트코인은 분산된 디지털'},
 {'description': '비트코인은 분산된 디지털 화'},
 {'description': '비트코인은 분산된 디지털 화폐'},
 {'description': '비트코인은 분산된 디지털 화폐로'},
 {'description': '비트코인은 분산된 디지털 화폐로,'},
 {'description': '비트코인은 분산된 디지털 화폐로, 중앙'},
 {'description': '비트코인은 분산된 디지털 화폐로, 중앙은행'},
 {'description': '비트코인은 분산된 디지털 화폐로, 중앙은행 없이'},
 {'description': '비트코인은 분산된 디지털 화폐로, 중앙은행 없이 개인'},
 {'description': '비트코인은 분산된 디지털 화폐로, 중앙은행 없이 개인 간'},
 {'description': '비트코인은 분산된 디지털 화폐로, 중앙은행 없이 개인 간의'},
 {'description': '비트코인은 분산된 디지털 화폐로, 중앙은행 없이 개인 간의 거래'},
 {'description': '비트코인은 분산된 디지털 화폐로, 중앙은행 없이 개인 간의 거래를'},
 {'description': '비트코인은 분산된 디지털 화폐로, 중앙은행 없이 개인 간의 거래를 가능'},
 {'description': '비트코인은 분산된 디지털 화폐로, 중앙은행 없이 개인 간의 거래를 가능하게'},
 {'description': '비트코인은 분산된 디

In [60]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
# 오픈AI 모델 설정
model = ChatOpenAI(temperature=0)


# 원하는 데이터 구조 정의
class FinancialAdvice(BaseModel):
    setup: str = Field(description="금융 조언 상황을 설정하기 위한 질문")
    advice: str = Field(description="질문을 해결하기 위한 금융 답변")

# JSON 출력 파서 설정 및 프롬프트 템플릿에 지침 삽입
parser = JsonOutputParser(pydantic_object=FinancialAdvice)
prompt = PromptTemplate(
    template="다음 금융 관련 질문에 답변해 주세요.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# 체인 구성: 프롬프트 -> 모델 -> 파서
chain = prompt | model | parser
# 체인 실행
chain.invoke({"query": "부동산에 관련하여 금융 조언을 받을 수 있는 질문하여라."})

{'setup': '부동산 투자에 대한 금융 조언을 받고 싶습니다.',
 'advice': '부동산 시장의 최신 동향을 파악하고 투자 전략을 세우는 것이 중요합니다.'}